# Demonstration for Ohio Model User's Group

In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import Project
from lasso import metcouncil

### Set up the Logger & Parameters

In [2]:
import logging
logger = logging.getLogger("WranglerLogger")
# logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Input/Output Locations & Parameters

In [3]:
input_dir = os.path.join("/Users", "wsp", "documents", "GitHub", "Lasso", "examples", "stpaul")
lasso_dir = os.path.join("/Users", "wsp", "documents", "GitHub", "Lasso")
output_dir = input_dir

stpaul_shape_file = os.path.join(input_dir, "shape.geojson")
stpaul_link_file = os.path.join(input_dir, "link.json")
stpaul_node_file = os.path.join(input_dir, "node.geojson")

In [4]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-02-03 12:43:29, INFO: Lasso base directory set as: /Users/wsp/documents/GitHub/Lasso


### Data Reads

In [5]:
roadway_net = RoadwayNetwork.read(
    link_file = stpaul_link_file, 
    node_file = stpaul_node_file, 
    shape_file = stpaul_shape_file, 
    fast = True        
)

transit_net = TransitNetwork.read(feed_path = input_dir)

2021-02-03 12:43:29, INFO: Reading from following files:
-/Users/wsp/documents/GitHub/Lasso/examples/stpaul/link.json
-/Users/wsp/documents/GitHub/Lasso/examples/stpaul/node.geojson
-/Users/wsp/documents/GitHub/Lasso/examples/stpaul/shape.geojson.
2021-02-03 12:43:34, INFO: Read 66253 links from /Users/wsp/documents/GitHub/Lasso/examples/stpaul/link.json
2021-02-03 12:43:34, INFO: Read 17159 nodes from /Users/wsp/documents/GitHub/Lasso/examples/stpaul/node.geojson
2021-02-03 12:43:34, INFO: Read 66253 shapes from /Users/wsp/documents/GitHub/Lasso/examples/stpaul/shape.geojson
2021-02-03 12:43:43, INFO: Read in transit feed from: /Users/wsp/documents/GitHub/Lasso/examples/stpaul
2021-02-03 12:43:43, INFO: Removing calendar.txt from transit network config because file not found
2021-02-03 12:43:43, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-02-03 12:43:43, INFO: Removing fare_attributes.txt from transit network config because file not found


### Make a Cube Network

In [6]:
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(roadway_net)
model_road_net = metcouncil.calculate_centroidconnect(model_road_net, parameters)
model_road_net.roadway_standard_to_met_council_network()

model_road_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, "links.txt"),
    output_node_txt = os.path.join(output_dir, "nodes.txt"),
    output_link_header_width_txt = os.path.join(output_dir, "links_header_width.txt"),
    output_node_header_width_txt = os.path.join(output_dir, "nodes_header_width.txt"),
    output_cube_network_script = os.path.join(output_dir, "make_complete_network_from_fixed_width_file.s"),
)

2021-02-03 12:43:53, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2021-02-03 12:43:53, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2021-02-03 12:43:53, INFO: Finished calculating centroid connector variable: centroidconnect
2021-02-03 12:43:53, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2021-02-03 12:43:53, INFO: Didn't detect managed lanes in network.
2021-02-03 12:43:53, INFO: Creating calculated roadway variables.
2021-02-03 12:43:53, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:423: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2021-02-03 12:44:05, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2021-02-03 12:44:05, INFO: Adding roadway network variable for county using a spatial join with: /Users/wsp/Documents/GitHub/Lasso/metcouncil_data/county/cb_2017_us_county_5m.shp


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2021-02-03 12:44:12, INFO: Finished Calculating county variable: county
2021-02-03 12:44:12, INFO: Calculating MPO as roadway network variable: mpo
2021-02-03 12:44:12, INFO: Finished calculating MPO variable: mpo
2021-02-03 12:44:12, INFO: Adding Counts
2021-02-03 12:44:12, INFO: Adding Variable AADT using Shared Streets Reference from /Users/wsp/Documents/GitHub/Lasso/metcouncil_data/count_mn/mn_count_ShSt_API_match.csv
2021-02-03 12:44:13, INFO: Added variable: AADT using Shared Streets Reference
2021-02-03 12:44:13, INFO: Adding Variable AADT using Shared Streets Reference from /Users/wsp/Documents/GitHub/Lasso/metcouncil_data/Wisconsin_Lanes_Counts_Median/wi_count_ShSt_API_match.csv
2021-02-03 12:44:13, INFO: Added variable: AADT using Shared Streets Reference
2021-02-03 12:44:13, INFO: Finished adding counts variable: AADT
2021-02-03 12:44:13, INFO: Finished creating ML lanes variable: ML_lanes
2021-02-03 12:44:13, INFO: Finished creating hov corridor variable: segment_id
2021-02

In [7]:
standard_transit = StandardTransit.fromTransitNetwork(transit_net)
standard_transit.write_as_cube_lin(outpath = os.path.join(output_dir, "transit.lin") )

2021-02-03 12:45:19, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2021-02-03 12:45:19, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard


In [8]:
## add step here to run cube making script

### Make a Project Card from a Cube Log File

In [9]:
project = Project.create_project(
        base_roadway_network = model_road_net,
        roadway_log_file = os.path.join(lasso_dir, "examples", "cube", "st_paul_test.log"),
        project_name = "Example",
    )

2021-02-03 12:45:19, INFO: No base transit network.
2021-02-03 12:45:19, INFO: No transit changes given or processed.
2021-02-03 12:45:19, INFO: Reading logfile: /Users/wsp/documents/GitHub/Lasso/examples/cube/st_paul_test.log
2021-02-03 12:45:19, INFO: Processed 1 Node lines and 7 Link lines
2021-02-03 12:45:19, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2021-02-03 12:45:19, INFO: Evaluating compatibility between roadway network changes and base network. Not evaluating deletions.
2021-02-03 12:45:19, INFO: Evaluating project changes.
2021-02-03 12:45:19, WARNING: The following attributes are specified in the changes but do not exist in the base network: ['OPERATION_final']
2021-02-03 12:45:19, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2021-02-03 12:45:19, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2021-02-03 12:45:19, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2021-02-03 12:45:19, 

/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/pandas/core/frame.py:1487: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [10]:
project.write_project_card(os.path.join(output_dir, "Example.yml"))

2021-02-03 12:45:19, INFO: Wrote project card to: /Users/wsp/documents/GitHub/Lasso/examples/stpaul/Example.yml


### Make a Scenario

In [11]:
base_scenario = {"road_net": roadway_net, "transit_net": transit_net}

In [12]:
build_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2021-02-03 12:45:19, INFO: Creating Scenario


In [13]:
#build_scenario.apply_project(project)
build_scenario.add_project_card_from_file(os.path.join(output_dir, "Example.yml"), validate = False)

In [14]:
build_scenario.applied_projects

[]

### Write Out Edited Cube Network

In [15]:
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(build_scenario.road_net)
model_road_net.roadway_standard_to_met_council_network()

model_road_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, "links.txt"),
    output_node_txt = os.path.join(output_dir, "nodes.txt"),
    output_link_header_width_txt = os.path.join(output_dir, "links_header_width.txt"),
    output_node_header_width_txt = os.path.join(output_dir, "nodes_header_width.txt"),
    output_cube_network_script = os.path.join(output_dir, "make_complete_network_from_fixed_width_file.s"),
)

2021-02-03 12:45:32, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2021-02-03 12:45:32, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2021-02-03 12:45:32, INFO: Creating managed lane network.
2021-02-03 12:45:32, INFO: Creating network with duplicated managed lanes
2021-02-03 12:45:33, INFO: Creating calculated roadway variables.
2021-02-03 12:45:33, INFO: Area Type Variable 'area_type' already in network. Returning without overwriting.
2021-02-03 12:45:33, INFO: County Variable 'county' already in network. Returning without overwriting.
2021-02-03 12:45:33, INFO: MPO Variable 'mpo' already in network. Returning without overwriting.
2021-02-03 12:45:33, INFO: Adding Counts
2021-02-03 12:45:33, INFO: Adding Variable AADT using Shared Streets Reference from /Users/wsp/Documents/GitHub/Lasso/metcouncil_data/count_mn/mn_count_ShSt_API_match.csv
2021-02-03 12:45:33, INFO: Added variable: AADT using Shared Streets Referenc